In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/cse519_project"

 all_feature.csv	      dataset.csv
 bots1.pgn.7z		      dataset_rob.csv
 bots.pgn.7z		      generating_datasets.ipynb
'CSE519-Final Report.gdoc'    lichess_db_standard_rated_2017-04.pgn.bz2
 Data_Science_Proposal.gdoc   lichess_db_standard_rated_2019-09.pgn.bz2
 dataset_2019.csv	      PreProcessing.ipynb


In [0]:
import pandas as pd
import bz2
import csv

In [0]:
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive.public

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 125048 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.5) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 163kB 19.6MB/s 


In [0]:
with open('/content/gdrive/My Drive/cse519_project/dataset_no_eval.csv', 'w', newline='') as csvfile:
  fieldnames = ['GAME_TYPE','WHITE_NAME','BLACK_NAME','RESULT','UTC_DATE','UTC_TIME', 'WHITE_ELO','BLACK_ELO','WHITE_RATING_DIFF', 'BLACK_RATING_DIFF','ECO','OPENING','TIME_CONTROL','TERMINATION','RECORD']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  
  writer.writeheader()

In [0]:
path = "/content/gdrive/My Drive/cse519_project/lichess_db_standard_rated_2017-04.pgn.bz2"
infile = bz2.BZ2File(path, "r")

while 1:
  try:
    line = infile.readline().decode('utf-8')
  except:
    break
  if 'Event' in line:
    if 'Blitz' in line:
      game_type = 'Blitz'
    elif 'Bullet' in line:
      game_type = 'Bullet'
    elif 'Classical' in line:
      game_type = 'Classical'
    # Site
    line = infile.readline().decode('utf-8')
    line = infile.readline().decode('utf-8')
    if 'White' in line:
      white_name=line[8:-3]
    line = infile.readline().decode('utf-8')
    if 'Black' in line:
      black_name = line[8:-3]
    line = infile.readline().decode('utf-8')
    if 'Result' in line:
      result = line[9:-3]
    line = infile.readline().decode('utf-8')
    if 'UTCDate' in line:
      utc_date = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'UTCTime' in line:
      utc_time = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'WhiteElo' in line:
        white_elo = line[11:-3]
    line = infile.readline().decode('utf-8')
    if 'BlackElo' in line:
      black_elo = line[11:-3]
    line = infile.readline().decode('utf-8')
    if 'WhiteRatingDiff' in line:
      white_rating_diff = line[18:-3]
    line = infile.readline().decode('utf-8')
    if 'BlackRatingDiff' in line:
      black_rating_diff = line[18:-3]
    line = infile.readline().decode('utf-8')
    if 'ECO' in line:
      eco = line[6:-3]
    line = infile.readline().decode('utf-8')
    if 'Opening' in line:
      opening = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'TimeControl' in line:
      time_control = line[14:-3]
    line = infile.readline().decode('utf-8')
    if 'Termination' in line:
      termination = line[14:-3]
    line = infile.readline().decode('utf-8')
    line = infile.readline().decode('utf-8')
    if '%eval' not in line:
      record = line
      newrow = [game_type, white_name, black_name, result, utc_date, 
                utc_time, white_elo, black_elo, white_rating_diff,
                black_rating_diff,eco, opening, time_control, termination, record]
      with open('/content/gdrive/My Drive/cse519_project/dataset_no_eval.csv', 'a+') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(newrow)
  else:
    continue


In [0]:
with open('/content/gdrive/My Drive/cse519_project/dataset_rob.csv', 'w', newline='') as csvfile:
  fieldnames = ['RESULT','WHITE_ELO','BLACK_ELO','RECORD']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()

In [0]:
with libarchive.public.file_reader('/content/gdrive/My Drive/cse519_project/bots1.pgn.7z') as reader:
  for entry in reader:
      with open("/tmp/{}".format(str(entry)), 'r') as f:
         for block in entry.get_blocks():
            record = (str(block)).split('\\r\\n')
            for  r in record:
              if 'Result' in r:
                  result = r.split('"')[1]
              elif 'WhiteElo' in r:
                  whiteELO = r.split('"')[1]
                  
              elif 'BlackElo' in r :
                  blackElo = r.split('"')[1]
              elif '}' in r:
                  try:
                    trpt = trpt + r
                  except:
                    trpt = r
                  if '1-0' in r or '0-1' in r or '1/2-1/2' in r:
                    transcript = trpt
                    del trpt
              try:
                  transcript
                  newrow = [result, whiteELO, blackElo, transcript]
                  with open('/content/gdrive/My Drive/cse519_project/dataset_rob.csv', 'a+') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(newrow)
                  del transcript
              except:
                  pass
                  


              
          

In [0]:
print("finish")

finish


In [0]:
with open('/content/gdrive/My Drive/cse519_project/dataset_no_eval.csv', 'w', newline='') as csvfile:
  fieldnames = ['GAME_TYPE','WHITE_NAME','BLACK_NAME','RESULT','UTC_DATE','UTC_TIME', 'WHITE_ELO','BLACK_ELO','WHITE_RATING_DIFF', 'BLACK_RATING_DIFF','ECO','OPENING','TIME_CONTROL','TERMINATION','RECORD']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  
  writer.writeheader()

In [0]:
path = "/content/gdrive/My Drive/cse519_project/lichess_db_standard_rated_2019-09.pgn.bz2"
infile = bz2.BZ2File(path, "r")

while 1:
  try:
    line = infile.readline().decode('utf-8')
  except:
    break
  if 'Event' in line:
    if 'Blitz' in line:
      game_type = 'Blitz'
    elif 'Bullet' in line:
      game_type = 'Bullet'
    elif 'Classical' in line:
      game_type = 'Classical'
    # Site
    line = infile.readline().decode('utf-8')
    line = infile.readline().decode('utf-8')
    if 'White' in line:
      white_name=line[8:-3]
    line = infile.readline().decode('utf-8')
    if 'Black' in line:
      black_name = line[8:-3]
    line = infile.readline().decode('utf-8')
    if 'Result' in line:
      result = line[9:-3]
    line = infile.readline().decode('utf-8')
    if 'UTCDate' in line:
      utc_date = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'UTCTime' in line:
      utc_time = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'WhiteElo' in line:
        white_elo = line[11:-3]
    line = infile.readline().decode('utf-8')
    if 'BlackElo' in line:
      black_elo = line[11:-3]
    line = infile.readline().decode('utf-8')
    if 'WhiteRatingDiff' in line:
      white_rating_diff = line[18:-3]
    line = infile.readline().decode('utf-8')
    if 'BlackRatingDiff' in line:
      black_rating_diff = line[18:-3]
    line = infile.readline().decode('utf-8')
    if 'ECO' in line:
      eco = line[6:-3]
    line = infile.readline().decode('utf-8')
    if 'Opening' in line:
      opening = line[10:-3]
    line = infile.readline().decode('utf-8')
    if 'TimeControl' in line:
      time_control = line[14:-3]
    line = infile.readline().decode('utf-8')
    if 'Termination' in line:
      termination = line[14:-3]
    line = infile.readline().decode('utf-8')
    line = infile.readline().decode('utf-8')
    if '%eval' in line:
      record = line
      newrow = [game_type, white_name, black_name, result, utc_date, 
                utc_time, white_elo, black_elo, white_rating_diff,
                black_rating_diff,eco, opening, time_control, termination, record]
      with open('/content/gdrive/My Drive/cse519_project/dataset_2019.csv', 'a+') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(newrow)
  else:
    continue


In [0]:
print("finish")

finish
